This NoteBook loades all data sets and saves them in six NumPy arrays for later quicker use and more balanced use
0=Angry, 1=Fear, 2=Happy, 3=Sad, 4=Surprise, 5=Neutral
We decided to lose the Disgusted emotion duo to irrelevance to our application goal

citings:
R Vemulapalli, A Agarwala, “A Compact Embedding for Facial Expression Similarity”, CoRR, abs/1811.11283, 2018.


#TO-DO

1.add database rafdb

#Imports and definitions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd                       #reading, writing and manipulating the data (using tables)
import numpy as np                        #Library for linear algebra and some probabiltity (raw data)
import tensorflow as tf                       
from keras.models import Sequential       #To create the sequential layer
from tensorflow.keras.callbacks import EarlyStopping

#from keras.layers.core import Flatten, Dense, Dropout     #To create the model
#from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D  #To create the model
from keras.preprocessing import image             #used for image classification
from keras.preprocessing.image import ImageDataGenerator  #used to expand the training dataset in order to improve the performance and ability of the model to generalize

from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.optimizers import SGD, Adam          #To use the optimizer
from keras.utils import np_utils  

#import cv2                                #CV (computer vision)
#from google.colab import files            #To be able to upload files

from PIL import Image,ImageFilter
stddev = 20
noise_filter = ImageFilter.GaussianBlur(radius=stddev)
emotions_counter = [0,0,0,0,0,0]

x = [[],[],[],[],[],[]]
y = [[],[],[],[],[],[]]
xk = [[],[],[],[],[],[]]
yk = [[],[],[],[],[],[]]
xf = [[],[],[],[],[],[]]
yf = [[],[],[],[],[],[]]



# Fer dataset

In [3]:
emotion_data = pd.read_csv('/content/drive/MyDrive/fer/fer2013.csv')
new_labels_data = pd.read_csv('/content/drive/MyDrive/fer/fer2013new.csv')

utilizing microsofts new labeling of fer2013 face repository to give better lables to faces

In [4]:
new_emotion = []
for i, row in new_labels_data.iterrows():
    emotion = max(row['neutral'],row['happiness'],row['surprise'],row['sadness'],row['anger'],row['disgust'],row['fear'])
    if (row['neutral']==emotion): max_index=5
    elif (row['happiness']==emotion): max_index=2
    elif (row['surprise']==emotion): max_index=4
    elif (row['sadness']==emotion): max_index=3
    elif (row['anger']==emotion): max_index=0
    elif (row['fear']==emotion): max_index=1
    else: max_index=10 #non relevent to model
    new_emotion.append(max_index)

Spliting the DataSet between faces to train on and faces to test accuracy with 

In [5]:

n=0
for index, row in emotion_data.iterrows():    #iterate on each row and store it
          emotion = new_emotion[index]            
          if emotion!=10:                         #10 is non relevent to model
              k = row['pixels'].split(" ")       #list of pixels, each pixel is an element    
              x[emotion].append(np.array(k,'float32'))    #The desired data-type for the array is 
              y[emotion].append(emotion)
          

            #-------------------------Noise tryout--------------------------#
                # # print('k')
                # # print(k)
                # image = Image.new(mode='L', size=(48, 48))
                # image.putdata(k)
                # noisy_image = image.filter(noise_filter)

                # # get the pixel data as a sequence
                # pixel_sequence = image.getdata()
                # print(pixel_sequence)
                # # convert the pixel sequence to a list
                # pixel_list = list(pixel_sequence)
                # # print('image')
                # print(pixel_list)
                # X_train.append(np.array(image,'float32'))    # Append the image array instead of k
                # y_train.append(new_emotion[index])


In [6]:
print(len(y[0]),len(x[0]))
print(len(y[1]),len(x[1]))
print(len(y[2]),len(x[2]))
print(len(y[3]),len(x[3]))
print(len(y[4]),len(x[4]))
print(len(y[5]),len(x[5]))


3148 3148
829 829
9377 9377
4438 4438
4501 4501
13318 13318


#KDEF dataset

In [7]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import load_img,img_to_array
from PIL import Image

for i in range(0,7):
      if(i!=1):
        # set the path to the folder of pictures
        folder_path = '/content/drive/MyDrive/kdef/'+str(i)

        # get a list of all the files in the folder
        pics = os.listdir(folder_path)
        
        # iterate through the image files
        for pic in pics:
              pic_dir = '/content/drive/MyDrive/kdef/'+str(i)+'/'+pic
              img_data = load_img(pic_dir, target_size = (48, 48))
              img_data = img_to_array(img_data)                            #convert the image to a Numpy array
              img_data = tf.image.rgb_to_grayscale(img_data)
              if(i==0):
                 yk[0].append(0)
                 xk[0].append(np.array(img_data,'float32'))    #The desired data-type for the array is
              else:  #other emotions need to shift one position left
                 yk[i-1].append(i-1)
                 xk[i-1].append(np.array(img_data,'float32'))    #The desired data-type for the array is
                

In [8]:
print(len(yk[0]),len(xk[0]))
print(len(yk[1]),len(xk[1]))
print(len(yk[2]),len(xk[2]))
print(len(yk[3]),len(xk[3]))
print(len(yk[4]),len(xk[4]))
print(len(yk[5]),len(xk[5]))


4724 4724
3454 3454
9049 9049
5403 5403
4226 4226
5071 5071


# FFHQ dataset

In [ ]:
!unzip /content/drive/MyDrive/ffhqlables/ffhq.zip

Streaming output truncated to the last 5000 lines.
 extracting: ffhq/65003.json         
 extracting: ffhq/65004.json         
 extracting: ffhq/65005.json         
 extracting: ffhq/65006.json         
 extracting: ffhq/65007.json         
 extracting: ffhq/65008.json         
 extracting: ffhq/65009.json         
 extracting: ffhq/65010.json         
 extracting: ffhq/65011.json         
 extracting: ffhq/65012.json         
 extracting: ffhq/65013.json         
 extracting: ffhq/65014.json         
 extracting: ffhq/65015.json         
 extracting: ffhq/65016.json         
 extracting: ffhq/65017.json         
 extracting: ffhq/65018.json         
 extracting: ffhq/65019.json         
 extracting: ffhq/65020.json         
 extracting: ffhq/65021.json         
 extracting: ffhq/65022.json         
 extracting: ffhq/65023.json         
 extracting: ffhq/65024.json         
 extracting: ffhq/65025.json         
 extracting: ffhq/65026.json         
 extracting: ffhq/65027.json         

In [ ]:
import os

file_names = os.listdir('/content/ffhq')
file_names_sorted = sorted(file_names)
for filename in file_names_sorted:
  if '.json' in filename:
    file_path = '/content/ffhq/'+filename
    file = open(file_path, "r")  # Open the JSON file for reading
    print(file.read())  # Read the JSON into the buffer
    file.close()  # Close the JSON file
   


In [ ]:
import os

y_trainffhq = []
file_names = os.listdir('/content/ffhq')
file_names_sorted = sorted(file_names)
for filename in file_names_sorted:
  if '.json' in filename:
    file_path = '/content/ffhq/'+filename
    file = open(file_path, "r")  # Open the JSON file for reading
    data = file.read()  # Read the JSON into the buffer
    file.close()  # Close the JSON file
    if ('neutral'==data): max_index=5
    elif ('happiness'==data): max_index=2
    elif ('surprise'==data): max_index=4
    elif ('sadness'==data): max_index=3
    elif ('anger'==data): max_index=0
    elif ('fear'==data): max_index=1
    else :max_index=10 #non relevent pictures
    y_trainffhq.append(max_index)
    print(max_index)


In [ ]:
print(len(y_trainffhq))

70000


In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import load_img,img_to_array
import json
from PIL import Image

# hc=0
i=0
for name in os.listdir('/content/drive/MyDrive/thumbnails128x128'):
        if os.path.isdir(os.path.join('/content/drive/MyDrive/thumbnails128x128/', name)):
            dir=(os.path.join('/content/drive/MyDrive/thumbnails128x128', name))
            # for each folder
            for img_name in os.listdir(dir):
                print(i)
                # #all of the cases which we dont add the picture
                if(y_trainffhq[i]!=10):
                      img_data = load_img(dir+'/'+img_name, target_size = (48, 48))        #load the image from the directory
                      img_data = img_to_array(img_data)
                      img_data = tf.image.rgb_to_grayscale(img_data)

                      emotion = y_trainffhq[i]                                     #current picture emotion type
                      xf[emotion].append(np.array(img_data,'float32'))    #appending picture to relevent X array
                      yf[emotion].append(emotion)                         
                i+=1

                        
                  
          

In [ ]:
print(len(yf[0]),len(xf[0]))
print(len(yf[1]),len(xf[1]))
print(len(yf[2]),len(xf[2]))
print(len(yf[3]),len(xf[3]))
print(len(yf[4]),len(xf[4]))
print(len(yf[5]),len(xf[5]))


289 289
26 26
43276 43276
652 652
835 835
24179 24179


#Reshaping arrays

In [10]:
#--------------------Convert Lists to Numpy arrays------------------------------
for i in range(0,6):
  x[i] = np.array(x[i], 'float32')
  y[i] = np.array(y[i], 'float32')
#-----------------------Reshape Pixels arrays---------------------------------
  #normalizing data between o and 1  
  x[i] -= np.mean(x[i], axis=0)  
  x[i] /= np.std(x[i], axis=0)
  #reshape the numpy array to be passed to the model
  x[i] = x[i].reshape(x[i].shape[0], 48, 48, 1)   
  #------------------Convert Labels array to categorial ones---------------------
  y[i]= np_utils.to_categorical(y[i], num_classes=6)



In [11]:
#--------------------Convert Lists to Numpy arrays------------------------------
for i in range(0,6):
  xk[i] = np.array(xk[i], 'float32')
  yk[i] = np.array(yk[i], 'float32')
#-----------------------Reshape Pixels arrays---------------------------------
  #normalizing data between o and 1  
  xk[i] -= np.mean(xk[i], axis=0)  
  xk[i] /= np.std(xk[i], axis=0)
  #reshape the numpy array to be passed to the model
  xk[i] = xk[i].reshape(xk[i].shape[0], 48, 48, 1)   
  #------------------Convert Labels array to categorial ones---------------------
  yk[i]= np_utils.to_categorical(yk[i], num_classes=6)

In [ ]:
# #--------------------Convert Lists to Numpy arrays------------------------------
# for i in range(0,6):
#   xf[i] = np.array(xf[i], 'float32')
#   yf[i] = np.array(yf[i], 'float32')
# #-----------------------Reshape Pixels arrays---------------------------------
#   #normalizing data between o and 1  
#   xf[i] -= np.mean(xf[i], axis=0)  
#   xf[i] /= np.std(xf[i], axis=0)
#   #reshape the numpy array to be passed to the model
#   xf[i] = xf[i].reshape(xf[i].shape[0], 48, 48, 1)   
#   #------------------Convert Labels array to categorial ones---------------------
#   yf[i]= np_utils.to_categorical(yf[i], num_classes=6)

# Saving the combined dataset

In [12]:
for i in range(0,6):
  print(x[i].shape,xk[i].shape)
  print(y[i].shape,yk[i].shape)


(3148, 48, 48, 1) (4724, 48, 48, 1)
(3148, 6) (4724, 6)
(829, 48, 48, 1) (3454, 48, 48, 1)
(829, 6) (3454, 6)
(9377, 48, 48, 1) (9049, 48, 48, 1)
(9377, 6) (9049, 6)
(4438, 48, 48, 1) (5403, 48, 48, 1)
(4438, 6) (5403, 6)
(4501, 48, 48, 1) (4226, 48, 48, 1)
(4501, 6) (4226, 6)
(13318, 48, 48, 1) (5071, 48, 48, 1)
(13318, 6) (5071, 6)


In [13]:
cx=[[],[],[],[],[],[]] #combined x
cy=[[],[],[],[],[],[]] #combined y
for i in range(0,6):
  # concatenate the the arrays vertically perserving their shape
  cx[i] = np.vstack((x[i],xk[i]))
  cy[i] = np.vstack((y[i],yk[i]))

  # print the shape of the combined array
  print(cx[i].shape)
  print(cy[i].shape)

(7872, 48, 48, 1)
(7872, 6)
(4283, 48, 48, 1)
(4283, 6)
(18426, 48, 48, 1)
(18426, 6)
(9841, 48, 48, 1)
(9841, 6)
(8727, 48, 48, 1)
(8727, 6)
(18389, 48, 48, 1)
(18389, 6)


In [15]:
for i in range(0,6):
  # Save arrays to disk
  np.save('/content/drive/MyDrive/cxcy/cx'+str(i)+'.npy', cx[i])
  np.save('/content/drive/MyDrive/cxcy/cy'+str(i)+'.npy', cy[i])

  # Load arrays from disk
  loaded_arr1 = np.load('/content/drive/MyDrive/cxcy/cx'+str(i)+'.npy')
  loaded_arr2 = np.load('/content/drive/MyDrive/cxcy/cy'+str(i)+'.npy')
  print(np.array_equal(cx[i], loaded_arr1))
  print(np.array_equal(cy[i], loaded_arr2))


True
True
True
True
True
True
True
True
True
True
True
True


In [16]:
for i in range(0,6):
  # Save arrays to disk
  np.save('/content/drive/MyDrive/cxcy/x'+str(i)+'.npy', x[i])
  np.save('/content/drive/MyDrive/cxcy/y'+str(i)+'.npy', y[i])
  np.save('/content/drive/MyDrive/cxcy/xk'+str(i)+'.npy', xk[i])
  np.save('/content/drive/MyDrive/cxcy/yk'+str(i)+'.npy', yk[i])

#Rafdb dataset

In [ ]:
!unzip /content/drive/MyDrive/rafdb.zip

In [ ]:
test_images_path =
train_images_path =
test_lables = pd.read_csv('/content/drive/MyDrive/fer/fer2013new.csv')
train_lables = pd.read_csv('/content/drive/MyDrive/fer/fer2013new.csv')

In [ ]:
new_emotion = []
for i, row in train_lables.iterrows():
    if(row['label'])==0:
      new_emotion.append(0)
    elif(row['label']==1):
      new_emotion.append(10)
    else:
      new_emotion.append(row['label'])-1)

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import load_img,img_to_array
import json
from PIL import Image

i=0
for name in os.listdir('/content/drive/MyDrive/thumbnails128x128'):
        if os.path.isdir(os.path.join('/content/drive/MyDrive/thumbnails128x128/', name)):
            dir=(os.path.join('/content/drive/MyDrive/thumbnails128x128', name))
            # for each folder
            for img_name in os.listdir(dir):
                print(i)
                # #all of the cases which we dont add the picture
                if(new_emotion[i]!=10):
                      img_data = load_img(dir+'/'+img_name, target_size = (48, 48))        #load the image from the directory
                      img_data = img_to_array(img_data)
                      img_data = tf.image.rgb_to_grayscale(img_data)

                      emotion = new_emotion[i]                                     #current picture emotion type
                      xf[emotion].append(np.array(img_data,'float32'))    #appending picture to relevent X array
                      yf[emotion].append(emotion)                         
                i+=1
i=0

for name in os.listdir('/content/drive/MyDrive/thumbnails128x128'):
        if os.path.isdir(os.path.join('/content/drive/MyDrive/thumbnails128x128/', name)):
            dir=(os.path.join('/content/drive/MyDrive/thumbnails128x128', name))
            # for each folder
            for img_name in os.listdir(dir):
                print(i)
                # #all of the cases which we dont add the picture
                if(new_emotion[i]!=10):
                      img_data = load_img(dir+'/'+img_name, target_size = (48, 48))        #load the image from the directory
                      img_data = img_to_array(img_data)
                      img_data = tf.image.rgb_to_grayscale(img_data)

                      emotion = new_emotion[i]                                     #current picture emotion type
                      xf[emotion].append(np.array(img_data,'float32'))    #appending picture to relevent X array
                      yf[emotion].append(emotion)                         
                i+=1

                        
                  
          

In [ ]:
print(len(y[0]),len(x[0]))
print(len(y[1]),len(x[1]))
print(len(y[2]),len(x[2]))
print(len(y[3]),len(x[3]))
print(len(y[4]),len(x[4]))
print(len(y[5]),len(x[5]))


3148 3148
829 829
9377 9377
4438 4438
4501 4501
13318 13318
